Source : https://martian1231-py.medium.com/ashrae-great-energy-predictor-iii-a-machine-learning-self-case-study-5c9e9d0ea11d

https://medium.com/analytics-vidhya/ashrae-great-energy-predictor-iii-a-machine-learning-case-study-a01a67eb048d

In [1]:
#installing streamlit and pyngrok
!pip -q install streamlit
!pip -q install pyngrok

In [2]:
from pyngrok import ngrok
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd
import numpy as np
import lightgbm as lgb
import pickle


In [3]:
%%writefile app.py

from pyngrok import ngrok
from sklearn.preprocessing import LabelEncoder
import joblib
import pandas as pd
import numpy as np
import streamlit as st
import lightgbm as lgb



file_to_be_upload= st.file_uploader("Select a file")
if file_to_be_upload is not None:

  test_point= pd.read_csv(file_to_be_upload)
  x_train= pd.read_feather('/content/drive/MyDrive/Ashrae_case_study/X_train_new.ftr')
  light_gbdt_model= joblib.load('lgbm_reg.sav')

  #handling all the missing values

  test_point['air_temperature'].fillna(0, inplace=True)
  test_point['cloud_coverage'].fillna(0, inplace=True)
  test_point['precip_depth_1_hr'].fillna(0, inplace=True)
  test_point['sea_level_pressure'].fillna(0, inplace=True)
  test_point['wind_direction'].fillna(0, inplace=True)
  test_point['wind_speed'].fillna(0, inplace=True)

  #Label encoding primary use and season features
  label_enc= LabelEncoder()
  label_enc.fit(x_train['primary_use'])
  test_point['primary_use']= label_enc.transform(test_point['primary_use'])

  label_enc.fit(x_train['season'])
  test_point['season']= label_enc.transform(test_point['season'])

  #predicting the data point
  log_energy_usage= light_gbdt_model.predict(test_point)
  energy_consumed= np.expm1(log_energy_usage)

  #display the predicted output in webpage
  st.subheader('Energy Prediction Using Ashrae Dataset:')
  st.write(energy_consumed)

Overwriting app.py


In [4]:
!ngrok authtoken 22QHtclLA01Usk575UMpbLtABkI_3yPVmGnUrynWTm3kYDoCA

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


**Creating Ngrok link**

In [5]:
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

NgrokTunnel: "http://ee7b-35-245-234-126.ngrok.io" -> "http://localhost:80"


# **Video link for my Deployment**

https://youtu.be/lNFvbbjGg_I